In [3]:
%pip install --upgrade pip
%pip install torch torchvision torchaudio
%pip install -q datasets jiwer
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post10-py3-none-any.whl size=2959 sha256=bc405c4cfd5cb2feb81ddde2e39458c0c68797968f1c7f9d6e1582ea2fbb53fa
  Stored in directory: /home/lxe/.cache/pip/wheels/5b/f6/92/0173054cc528db7ffe7b0c7652a96c3102aab156a6da960387
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, sys, itertools
os.environ['TOKENIZERS_PARALLELISM']='false'

import pandas as pd
from sklearn.model_selection import train_test_split

from PIL import Image

import torch
from torch.utils.data import Dataset

import datasets
from datasets import load_dataset

import transformers
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import VisionEncoderDecoderModel, TrOCRProcessor, default_data_collator

import evaluate

/home/lxe/miniconda3/envs/lpread/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print("Python:".rjust(15), sys.version[0:6])
print("Pandas:".rjust(15), pd.__version__)
print("Datasets:".rjust(15), datasets.__version__)
print("Transformers:".rjust(15), transformers.__version__)
print("Torch:".rjust(15), torch.__version__)

        Python: 3.10.1
        Pandas: 2.1.2
      Datasets: 2.14.6
  Transformers: 4.35.0
         Torch: 2.1.0+cu118


In [4]:
path = "plates.csv"

dataset = pd.read_csv(path)
dataset.rename(columns={"images" : "file_name", "labels" : "text"}, inplace=True)

# train/test split
train_dataset, test_dataset = train_test_split(dataset, train_size=0.80, random_state=42)

train_dataset.reset_index(drop=True, inplace=True)
test_dataset.reset_index(drop=True, inplace=True)

print(train_dataset.info())
print(test_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   83 non-null     object
 1   label   83 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   21 non-null     object
 1   label   21 non-null     object
dtypes: object(2)
memory usage: 464.0+ bytes
None


In [5]:
train_dataset.head(12)

,image,label
0,plates/191_0.jpg,9GLW121
1,plates/57_0.jpg,KG004
2,plates/265_0.jpg,6AKK175
3,plates/174_0.jpg,6SRW765
4,plates/249_0.jpg,9FIR383
5,plates/36_0.jpg,6CRR781
6,plates/172_0.jpg,6SRW765
7,plates/121_0.jpg,7SHG843
8,plates/27_0.jpg,7HGH819
9,plates/314_0.jpg,6FXK819


In [6]:
class License_Plates_OCR_Dataset(Dataset):
    
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, padding="max_length", max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id 
                  else -100 for label in labels]
        
        encoding = {"pixel_values" : pixel_values.squeeze(), "labels" : torch.tensor(labels)}
        return encoding

In [7]:
MODEL_CKPT = "microsoft/trocr-base-printed"
MODEL_NAME =  MODEL_CKPT.split("/")[-1] + "_license_plates_ocr"
NUM_OF_EPOCHS = 2

In [8]:
processor = TrOCRProcessor.from_pretrained(MODEL_CKPT)

cwd = os.getcwd()

train_ds = License_Plates_OCR_Dataset(root_dir=cwd, df=train_dataset, processor=processor)
test_ds = License_Plates_OCR_Dataset(root_dir=cwd, df=test_dataset, processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


TypeError: License_Plates_OCR_Dataset.__init__() got an unexpected keyword argument 'rocessor'